In [6]:
from __future__ import print_function

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop, Adadelta, Adagrad, Adam, Nadam, SGD
from keras.callbacks import EarlyStopping, TerminateOnNaN
from keras import backend as K
from keras.losses import mean_squared_error
import tensorflow as tf

# Normalize the data.
from sklearn import preprocessing
from keras.regularizers import l1_l2

import random

def pass_arg(nsim, tr_size, dropoutrate):
    print("Tr_size:", tr_size)
    def fix_seeds(seed):
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
        session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
        sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
    #     K.set_session(sess)
        tf.compat.v1.keras.backend.set_session(sess)

    ss = 1
    fix_seeds(ss)

    # MC dropout
    class MCDropout(Dropout):
        def call(self, inputs, training=None):
            return super(MCDropout, self).call(inputs, training=True)

    # import pickle

    # def save_obj(obj, name):
    #     with open(name, 'wb') as f:
    #         pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

    
    # Compute the RMSE given the ground truth (y_true) and the predictions(y_pred)
    def root_mean_squared_error(y_true, y_pred):
            return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 


    # Making sure final porosity is less than initial
    def poros(poroi, porof):
        return K.relu(tf.negative(porof)) + K.relu(porof-poroi)

    def phy_loss_mean(params):
        # useful for cross-checking training
        loss1, lam1 = params
        def loss(y_true,y_pred):
            return lam1*K.mean(K.relu(loss1))
        return loss

    #function to calculate the combined loss = sum of rmse and phy based loss
    def combined_loss(params):
        loss1, lam1 = params
        def loss(y_true,y_pred):
            return mean_squared_error(y_true, y_pred) + lam1 * K.mean(K.relu(loss1))
        return loss

    def PGNN_train_test(optimizer_name, optimizer_val, drop_frac, use_YPhy, iteration, n_layers, n_nodes, tr_size, lamda, reg, samp):

    #     fix_seeds(ss)

        # Hyper-parameters of the training process
    #     batch_size = tr_size
        batch_size = 10
        num_epochs = 300
        val_frac = 0.25
        patience_val = 80

        # Initializing results filename
        exp_name = "DNN_loss" + optimizer_name + '_drop' + str(drop_frac) + '_usePhy' + str(use_YPhy) +  '_nL' + str(n_layers) + '_nN' + str(n_nodes) + '_trsize' + str(tr_size) + '_lamda' + str(lamda) + '_iter' + str(iteration)
        exp_name = exp_name.replace('.','pt')
        results_dir = '../results/'
        model_name = results_dir + exp_name + '_model.h5' # storing the trained model

        if reg==True and samp==25:
            results_name = results_dir + exp_name + '_results_25_regularizer.dat' # storing the results of the model
        elif reg==False and samp==25:
            results_name = results_dir + exp_name + '_results_25.dat' # storing the results of the model
        elif reg==True and samp==1519:
            results_name = results_dir + exp_name + '_results_1519_regularizer.dat' # storing the results of the model
        elif reg==False and samp==1519:
            results_name = results_dir + exp_name + '_results_1519.dat' # storing the results of the model

        # Load labeled data
        data = np.loadtxt('../data/labeled_data.dat')
    #     data = np.loadtxt('../data/labeled_data_BK_constw_unique.dat')
    #     data = np.loadtxt('../data/labeled_data_BK_constw_v2.dat')
        x_labeled = data[:, :2] # -2 because we do not need porosity predictions
        y_labeled = data[:, -2:-1] # dimensionless bond length and porosity measurements
        if samp==25:
            data = np.loadtxt('../data/unlabeled_data_BK_constw_v2_25.dat')
            x_unlabeled = data[:, :]
        elif samp==1519:
            data = np.loadtxt('../data/unlabeled_data_BK_constw_v2_1525.dat')
            x_unlabeled = data[:, :]

        x_unlabeled1 = x_unlabeled[:1303, :]
        x_unlabeled2 = x_unlabeled[-6:, :]
        x_unlabeled = np.vstack((x_unlabeled1,x_unlabeled2))

        # initial porosity
        init_poro = x_unlabeled[:, -1]
        x_unlabeled = x_unlabeled[:, :2]

    #     data = np.loadtxt('../data/unlabeled_data_BK_constw_v2_1519.dat')
    #     x_unlabeled = data[:1303, :] # 1303 last regular sample: 260, 46
    #     x_unlabeled_non = x_unlabeled



        # normalize dataset with MinMaxScaler
        scaler = preprocessing.MinMaxScaler(feature_range=(0.0, 1.0))
    #     scaler = preprocessing.StandardScaler()
        x_labeled = scaler.fit_transform(x_labeled)
        # y_labeled = scaler.fit_transform(y_labeled)
        x_unlabeled = scaler.fit_transform(x_unlabeled)

    #     # initial porosity & physics outputs are removed
    #     x_unlabeled = x_unlabeled[:, :-3]

        # train and test data
        trainX, trainY = x_labeled[:tr_size,:], y_labeled[:tr_size]
    #     testX, testY = x_labeled[tr_size:,:], y_labeled[tr_size:]   
#         testX, testY = x_labeled[tr_size:,:], y_labeled[tr_size:]
        testX, testY = x_labeled[30:,:], y_labeled[30:]

        if use_YPhy == 0:
            # Removing the last column from x_unlabeled (corresponding to Y_PHY)
            x_unlabeled = x_unlabeled[:,:-1]

        # Creating the model
        model = Sequential()
        for layer in np.arange(n_layers):
            if layer == 0:
                model.add(Dense(n_nodes, activation='relu', input_shape=(np.shape(trainX)[1],)))
            else:
                if reg:
                    model.add(Dense(n_nodes, activation='relu', kernel_regularizer=l1_l2(l1=.001, l2=.001)))
                else:
                    model.add(Dense(n_nodes, activation='relu'))
            # model.add(Dropout(rate=drop_frac))
            model.add(MCDropout(rate=drop_frac))
        model.add(Dense(1, activation='linear'))

        # physics-based regularization
        uinp_sc = K.constant(value=x_unlabeled) # unlabeled input data
        lam1 = K.constant(value=lamda) # regularization hyper-parameter
        predictions = model(uinp_sc) # model output at depth i
        phyloss2 = poros(init_poro, predictions) # physics loss 1
        totloss = combined_loss([phyloss2, lam1])
        phyloss = phy_loss_mean([phyloss2, lam1])
		
        model.compile(loss=totloss,
                      optimizer=optimizer_val,
                      metrics=[phyloss, root_mean_squared_error])

        early_stopping = EarlyStopping(monitor='val_loss', patience=patience_val, verbose=1)

    #     print('Running...' + optimizer_name)
        history = model.fit(trainX, trainY,
                            batch_size=batch_size,
                            epochs=num_epochs,
                            verbose=0,
                            validation_split=val_frac, callbacks=[early_stopping, TerminateOnNaN()])
    
        test_score = model.evaluate(testX, testY, verbose=1)
        print(test_score)

        test_scores = []
        for i in range(int(nsim)):
#             print("simulation num:",i)
#             predictions = model.predict(testX)
#             samples.append(predictions)
            test_score = model.evaluate(testX, testY, verbose=0)
            test_scores.append(test_score[2])
        return np.array(test_scores)
    
    



    # Main Function
    if __name__ == '__main__':

        fix_seeds(1)

        # List of optimizers to choose from    
        optimizer_names = ['Adagrad', 'Adadelta', 'Adam', 'Nadam', 'RMSprop', 'SGD', 'NSGD']
        optimizer_vals = [Adagrad(clipnorm=1), Adadelta(clipnorm=1), Adam(clipnorm=1), Nadam(clipnorm=1), RMSprop(clipnorm=1), SGD(clipnorm=1.), SGD(clipnorm=1, nesterov=True)]

        # selecting the optimizer
        optimizer_num = 1
        optimizer_name = optimizer_names[optimizer_num]
        optimizer_val = optimizer_vals[optimizer_num]

        # Selecting Other Hyper-parameters
        drop_frac = dropoutrate # Fraction of nodes to be dropped out
        use_YPhy = 1 # Whether YPhy is used as another feature in the NN model or not
        n_layers = 2 # Number of hidden layers
        n_nodes = 5 # Number of nodes per hidden layer

        #set lamda
        lamda = 0.01 # Physics-based regularization constant  

#         # Iterating over different training fractions and splitting indices for train-test splits
#         trsize_range = [4,6,8,10,20]

#         #default training size = 5000
#         tr_size = trsize_range[4]
        
        tr_size = int(tr_size)

        # use regularizer
        reg = True

        # sample size used
        samp = 1519
    #     samp = 25

        # total number of runs
        iter_range = np.arange(1)
        testrmse=[]
        # iterating through all possible params
        for iteration in iter_range:
#             results, result_file, pred, obs, rmse, obs_train = PGNN_train_test(optimizer_name, optimizer_val, drop_frac, use_YPhy, 
#                             iteration, n_layers, n_nodes, tr_size, lamda, reg, samp)
#             testrmse.append(rmse)
            pred = PGNN_train_test(optimizer_name, optimizer_val, drop_frac, use_YPhy, 
                            iteration, n_layers, n_nodes, tr_size, lamda, reg, samp)
            

    return np.squeeze(pred)

In [7]:
mean_rmses=[]
std_rmses=[]
# for ii in ([.005,.01,.02,.05,.1,.15,.2,.25,.3,0.5]):
for ii in ([5,10,15,20,30]):
    test_rmse = pass_arg(50, ii, 0.05)
    mean_rmse = np.mean(test_rmse)
    std_rmse = np.std(test_rmse)
    mean_rmses.append(mean_rmse)
    std_rmses.append(std_rmse)

Tr_size: 5
9/9 [==============================] - 0s 666us/step
[0.005317178089171648, 2.7532059903023764e-05, 0.009709746576845646]
Tr_size: 10
9/9 [==============================] - 0s 665us/step
[0.005151878111064434, 2.4918010240071453e-05, 0.014143387787044048]
Tr_size: 15
9/9 [==============================] - 0s 997us/step
[0.001799504505470395, 1.3287068213685416e-05, 0.007933520711958408]
Tr_size: 20
9/9 [==============================] - 0s 779us/step
[0.0013390794629231095, 9.434725143364631e-07, 0.011573431082069874]
Tr_size: 30
9/9 [==============================] - 0s 772us/step
[0.0009491285891272128, 2.781316155164859e-08, 0.017200473695993423]


In [8]:
mean_rmses

[0.017790529355406762,
 0.015012979414314032,
 0.01397161060012877,
 0.01454856589436531,
 0.013390020728111266]

In [9]:
std_rmses

[0.00736550871147045,
 0.00825442648135047,
 0.0058420587771927535,
 0.002808790887817009,
 0.002428422070285528]

In [34]:
import pickle

def save_obj(obj, name):
    with open(name, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
        
save_obj(mean_rmses, "../mean_rmse_dnn_loss_MC.dat")
save_obj(std_rmses, "../std_rmse_dnn_loss_MC.dat")


In [3]:
mc_pred=np.mean(pred,axis=0)
mc_pred

array([0.05279518, 0.03283933, 0.0537447 , 0.03974609, 0.03457924,
       0.03742982, 0.04465229, 0.05489689, 0.0322278 , 0.04860931,
       0.05708254, 0.04503478, 0.06214603, 0.04120987, 0.04285658,
       0.03374553, 0.04266082, 0.04460517, 0.04174339, 0.0369988 ,
       0.06102522, 0.04720526, 0.03547252, 0.04057915, 0.06028698,
       0.04594869, 0.03679036, 0.04504748, 0.05434823, 0.05183214,
       0.04711368, 0.03764421, 0.03771775, 0.05239221, 0.04771913,
       0.04199286, 0.05519148, 0.04541095, 0.06952468, 0.04033285,
       0.04858945, 0.03479638, 0.03056115, 0.03366404, 0.05692469,
       0.05566785, 0.0399124 , 0.04820377, 0.04281671, 0.03658003,
       0.05378899, 0.04206469, 0.04401018, 0.03655737, 0.06820614,
       0.05063169, 0.04610386, 0.05112227, 0.0458006 , 0.04457092,
       0.03266823, 0.04351811, 0.03674877, 0.03626641, 0.06036571,
       0.04281345, 0.04486926, 0.05853532, 0.0412476 , 0.03061781,
       0.03981644, 0.04545145, 0.04714443, 0.0347507 , 0.04353

In [4]:
import pickle

def save_obj(obj, name):
    with open(name, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
        
save_obj(mc_pred, "../pred_loss_MC_Xx.dat")